In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# df_4 = pd.read_csv('/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-10/csv_final/pan_mit_b3_ttaV2.csv')
df_3 = pd.read_csv('/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-10/csv_final/hard_voting_5models.csv')
df_1 = pd.read_csv('/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-10/csv_final/knet_soup.csv')
df_2 = pd.read_csv('/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-10/csv_final/pan_swin_ens.csv')

In [5]:
PredictionString = []

for idx in tqdm(range(len(df_1))):
    aa = df_1['PredictionString'][idx].split(' ')
    bb = df_2['PredictionString'][idx].split(' ')
    cc = df_3['PredictionString'][idx].split(' ')
    # dd = df_4['PredictionString'][idx].split(' ')
    
    result = ''

    for i in range(len(aa)):
        pixel_dict = {'0' : 0, '1' : 0, '2' : 0, 
                      '3' : 0, '4' : 0, '5' : 0,
                      '6' : 0, '7' : 0, '8' : 0,
                      '9' : 0, '10' : 0}

        best_model = aa[i]
        second_model = bb[i]
        third_model = cc[i]
        
        pixel_dict[aa[i]] += 1
        pixel_dict[bb[i]] += 1
        pixel_dict[cc[i]] += 1
        # pixel_dict[dd[i]] += 1

        # 최댓값을 갖는 키값 추출
        pixel = [key for key, value in pixel_dict.items() if value == max(pixel_dict.values())]
        
        # ensemble 모델이 ~4개
        # 만약 최댓값을 갖는 키가 2개 이상(같은 표를 가진 경우)이고,
        # best_model이 영향력을 행사했다면
        if len(pixel) >= 2 and best_model in pixel:
            result += best_model + ' '
        
        # ensemble 모델이 5개~6개
        # 만약 최댓값을 갖는 키가 2개 이상(같은 표를 가진 경우)이고,
        # best_model이 outsider, second_model이 영향력을 행사했다면
        elif len(pixel) >= 2 and second_model in pixel:
            result += second_model + ' '
            
        # 만약 1개라면 해당 키 반환
        else:
            result += pixel[0] + ' '

    # 마지막 공백 제거
    result = result[:-1]
    PredictionString.append(result)


100%|██████████| 819/819 [06:42<00:00,  2.03it/s]


In [6]:
submission = pd.DataFrame()
submission['image_id'] = df_1['image_id']
submission['PredictionString'] = PredictionString

In [7]:
submission.to_csv('/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-10/csv_final/hard_voting_models_v2.csv', index=False)